# Importing data from Kaggle

##Train and validation data

In [ ]:
!pip install -q kaggle
!pip install -q kaggle-cli
!mkdir -p ~/.kaggle
!cp "/content/drive/MyDrive/kaggle.json" ~/.kaggle/
!cat ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vaishnaviasonawane/indian-sign-language-dataset

In [ ]:
import zipfile
local_zip = '/content/indian-sign-language-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

##Test Data

In [ ]:
!kaggle datasets download -d prathumarikeri/indian-sign-language-isl

import zipfile
local_zip = '/content/indian-sign-language-isl.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

 99% 278M/281M [00:10<00:00, 35.9MB/s]
100% 281M/281M [00:10<00:00, 28.2MB/s]


#Splitting data into train and validation


In [ ]:
import os
import numpy as np
import shutil

rootdir= 'data'

classes = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

for i in classes:
  os.makedirs(rootdir +'/train/' + i)
  os.makedirs(rootdir +'/valid/' + i)
  source = rootdir + '/' + i

  allFileNames = os.listdir(source)
  np.random.shuffle(allFileNames)
  val_ratio = 0.3
  train_FileNames, val_FileNames = np.split(np.array(allFileNames), [int(len(allFileNames)* (1 - val_ratio))])

  train_FileNames = [source+'/'+ name for name in train_FileNames.tolist()]
  val_FileNames = [source+'/' + name for name in val_FileNames.tolist()]

  for name in train_FileNames:
    shutil.copy(name, rootdir +'/train/' + i)

  for name in val_FileNames:
    shutil.copy(name, rootdir +'/valid/' + i)

#Models

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_dir = "data/train"
validation_dir = "data/valid"
test_dir = "Indian"

In [ ]:
import tensorflow as tf

from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet_v2 import ResNet50V2

In [ ]:
models = [InceptionV3, ResNet50V2]

## InceptionV3

In [ ]:
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(299, 299),
batch_size=128,
class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
validation_dir,
target_size=(299, 299),
batch_size=32,
class_mode='categorical')

base_model = models[0](input_shape=(299,299,3), weights='imagenet', include_top=False)
base_model.trainable = False

inc = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(35, activation='softmax')
])
inc.summary()

inc.compile(loss = 'categorical_crossentropy', metrics = [['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]])
inc1 = inc.fit(train_generator, epochs=2, validation_data=validation_generator)

Found 29400 images belonging to 35 classes.
Found 12600 images belonging to 35 classes.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 flatten_2 (Flatten)         (None, 131072)            0         
                                                                 
 dense_2 (Dense)             (None, 35)                4587555   
                                                                 
Total params: 26,390,339
Trainable params: 4,587,555
Non-trainable params: 21,802,784
_________________________________________________________________
Epoch 1/2
230/230 [==============================] - 166s 702ms/step - loss: 0.2519 - acc: 0.9907 - precision_2: 0.9949 - recall_2: 0.9907 - auc_2: 0.9975 - val_loss: 1.9289e-08 - val_acc: 1.0000 - val_precisio

In [ ]:
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size=(299, 299),
batch_size=128,
class_mode='categorical')

inc.evaluate(test_generator)

Found 42745 images belonging to 35 classes.
334/334 [==============================] - 165s 492ms/step - loss: 0.7354 - acc: 0.9875 - precision_2: 0.9875 - recall_2: 0.9875 - auc_2: 0.9937


[0.7354331612586975,
 0.9875307083129883,
 0.9875307083129883,
 0.9875307083129883,
 0.9936813116073608]

##ResNet50V2

In [ ]:
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(224, 224),
batch_size=128,
class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
validation_dir,
target_size=(224, 224),
batch_size=32,
class_mode='categorical')

base_model = models[1](input_shape=(224,224,3), weights='imagenet', include_top=False)
base_model.trainable = False

res = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(35, activation='softmax')
])
res.summary()

res.compile(loss = 'categorical_crossentropy', metrics = [['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]])
res1 = res.fit(train_generator, epochs=2, validation_data=validation_generator)

Found 29400 images belonging to 35 classes.
Found 12600 images belonging to 35 classes.
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten_3 (Flatten)         (None, 100352)            0         
                                                                 
 dense_3 (Dense)             (None, 35)                3512355   
                                                                 
Total params: 27,077,155
Trainable params: 3,512,355
Non-trainable params: 23,564,800
_________________________________________________________________
Epoch 1/2
230/230 [==============================] - 121s 514ms/step - loss: 0.1013 - acc: 0.9938 - precision_3: 0.9967 - recall_3: 0.9937 - auc_3: 0.9978 - val_loss: 7.6634e-10 - val_acc: 1.0000 - val_precisio

In [ ]:
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size=(224, 224),
batch_size=128,
class_mode='categorical')

res.evaluate(test_generator)

Found 42745 images belonging to 35 classes.
334/334 [==============================] - 119s 355ms/step - loss: 0.2589 - acc: 0.9868 - precision_3: 0.9875 - recall_3: 0.9868 - auc_3: 0.9941


[0.2588743567466736,
 0.9868288636207581,
 0.9874750971794128,
 0.9867820739746094,
 0.9941183924674988]

#Saving the models

In [ ]:
inc.save("/content/drive/MyDrive/DL_MP/inc/inc.h5")
res.save("/content/drive/MyDrive/DL_MP/res/res.h5")